# IMDB Dataset

Dataset of 25,000 movies reviews from IMDB, labeled by sentiment (positive/negative). 

Reviews have been preprocessed, and each review is encoded as a sequence of word indexes (integers). 

For convenience, words are indexed by overall frequency in the dataset, so that for instance the integer "3" encodes the 3rd most frequent word in the data. This allows for quick filtering operations such as: "only consider the top 10,000 most common words, but eliminate the top 20 most common words".

As a convention, "0" does not stand for a specific word, but instead is used to encode any unknown word.



In [1]:
from keras.datasets import imdb
top_words = 5000
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=top_words)
print((x_train[0]))


Using TensorFlow backend.


 4595712/17464789 [======>.......................] - ETA: 15s

KeyboardInterrupt: 

# Getting the original review from word index


start_char: int. The start of a sequence will be marked with this character. Set to 1 because 0 is usually the padding character.

oov_char: int. words that were cut out because of the num_words or skip_top limit will be replaced with this character.

index_from: int. Index actual words with this index and higher.

indices returned by keras have `START` and `UNKNOWN` as indexes 1 and 2. (And it assumes you will use 0 for `PADDING`).

In [46]:
import keras
NUM_WORDS=1000 # only use top 1000 words
INDEX_FROM=3   # word index offset

train,test = keras.datasets.imdb.load_data(num_words=NUM_WORDS, index_from=INDEX_FROM)
train_x,train_y = train
test_x,test_y = test

word_to_id = keras.datasets.imdb.get_word_index()
word_to_id = {k:(v+INDEX_FROM) for k,v in word_to_id.items()}
word_to_id["<PAD>"] = 0
word_to_id["<START>"] = 1
word_to_id["<UNK>"] = 2

id_to_word = {value:key for key,value in word_to_id.items()}
print(' '.join(id_to_word[id] for id in train_x[0]))

1556480/1641221 [===========================>..] - ETA: 0s<START> this film was just brilliant casting <UNK> <UNK> story direction <UNK> really <UNK> the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same <UNK> <UNK> as myself so i loved the fact there was a real <UNK> with this film the <UNK> <UNK> throughout the film were great it was just brilliant so much that i <UNK> the film as soon as it was released for <UNK> and would recommend it to everyone to watch and the <UNK> <UNK> was amazing really <UNK> at the end it was so sad and you know what they say if you <UNK> at a film it must have been good and this definitely was also <UNK> to the two little <UNK> that played the <UNK> of <UNK> and paul they were just brilliant children are often left out of the <UNK> <UNK> i think because the stars that play them all <UNK> up are such a big <UNK> for the whole film but these children are am

# Using CNN

In [4]:
from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.datasets import imdb


In [5]:
# set parameters:
top_words = 5000
maxlen = 400
batch_size = 32
embedding_dims = 50
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 2

from keras.datasets import imdb

(x_train, y_train), (x_test, y_test) = imdb.load_data(path="imdb.npz",
                                                      num_words=None,
                                                      skip_top=0,
                                                      maxlen=None,
                                                      seed=113,
                                                      start_char=1,
                                                      oov_char=2,
                                                      index_from=3)

Arguments:

path: if you do not have the data locally (at '~/.keras/datasets/' + path), it will be downloaded to this location.
num_words: integer or None. Top most frequent words to consider. Any less frequent word will appear as oov_char value in the sequence data.

skip_top: integer. Top most frequent words to ignore (they will appear as oov_char value in the sequence data).

maxlen: int. Maximum sequence length. Any longer sequence will be truncated.

seed: int. Seed for reproducible data shuffling.

start_char: int. The start of a sequence will be marked with this character. Set to 1 because 0 is usually the padding character.

oov_char: int. words that were cut out because of the num_words or skip_top limit will be replaced with this character.

index_from: int. Index actual words with this index and higher.

In [8]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=top_words)
print(len(x_train), 'train sequences')


25000 train sequences


In [9]:
print(len(x_test), 'test sequences')


25000 test sequences


In [10]:
print('x_train shape:', x_train.shape)


x_train shape: (25000,)


In [11]:
print('x_test shape:', x_test.shape)


x_test shape: (25000,)


In [12]:
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)


x_train shape: (25000, 400)
x_test shape: (25000, 400)


### Global max pooling 

For example, if the input of the max pooling layer is 0,1,2,2,5,1,20,1,2,2,5,1,2, global max pooling outputs 5, whereas ordinary max pooling layer with pool size equals to 3 outputs 2,2,5,5,5,2,2,5,5,5 (assuming stride=1).

In some domains, such as natural language processing, it is common to use global max pooling. In some other domains, such as computer vision, it is common to use a max pooling that isn't global.

In [13]:
model = Sequential()

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
model.add(Embedding(top_words,
                    embedding_dims,
                    input_length=maxlen))
model.add(Dropout(0.2))

# we add a Convolution1D, which will learn filters
# word group filters of size filter_length:
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
# we use max pooling:
model.add(GlobalMaxPooling1D())

# We add a vanilla hidden layer:
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))

# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 387s - loss: 0.4133 - acc: 0.7978 - val_loss: 0.2966 - val_acc: 0.8726
Epoch 2/2
25000/25000 [==============================] - 393s - loss: 0.2355 - acc: 0.9036 - val_loss: 0.2754 - val_acc: 0.8862


# Using LSTMs

In [27]:
from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb

top_words = 20000
maxlen = 80  # cut texts after this number of words (among top max_features most common words)
batch_size = 32
embedding_dims = 128

In [28]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=top_words)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

25000 train sequences
25000 test sequences


In [29]:
print('x_train shape:', x_train.shape)

x_train shape: (25000,)


In [30]:
print('x_test shape:', x_test.shape)

x_test shape: (25000,)


In [31]:
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train shape: (25000, 80)
x_test shape: (25000, 80)


In [33]:
model = Sequential()
model.add(Embedding(top_words, embedding_dims, input_length=maxlen))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=2,
          validation_data=(x_test, y_test))

Train...
Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 418s - loss: 0.4615 - acc: 0.7833 - val_loss: 0.3663 - val_acc: 0.8398
Epoch 2/2
25000/25000 [==============================] - 438s - loss: 0.2988 - acc: 0.8808 - val_loss: 0.4538 - val_acc: 0.8130


# Using CNN + LSTMS

In [34]:
from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import Conv1D, MaxPooling1D
from keras.datasets import imdb

In [35]:
# Embedding
top_words = 20000
maxlen = 100
embedding_size = 128

# Convolution
kernel_size = 5
filters = 64
pool_size = 4

# LSTM
lstm_output_size = 70

# Training
batch_size = 30
epochs = 2


In [39]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=top_words)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

25000 train sequences
25000 test sequences


In [40]:
print('x_train shape:', x_train.shape)

x_train shape: (25000,)


In [41]:
print('x_test shape:', x_test.shape)

x_test shape: (25000,)


In [42]:
print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Pad sequences (samples x time)
x_train shape: (25000, 100)
x_test shape: (25000, 100)


In [43]:
model = Sequential()
model.add(Embedding(top_words, embedding_size, input_length=maxlen))
model.add(Dropout(0.25))
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
model.add(MaxPooling1D(pool_size=pool_size))
model.add(LSTM(lstm_output_size))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test))

Train...
Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 228s - loss: 0.3859 - acc: 0.8166 - val_loss: 0.3475 - val_acc: 0.8526
Epoch 2/2
25000/25000 [==============================] - 235s - loss: 0.1985 - acc: 0.9242 - val_loss: 0.3865 - val_acc: 0.8520


# Using BiDirectional LSTMs

LSTM in its core, preserves information from inputs that has already passed through it using the hidden state.

Unidirectional LSTM only preserves information of the past because the only inputs it has seen are from the past.

Using bidirectional will run your inputs in two ways, one from past to future and one from future to past and what differs this approach from unidirectional is that in the LSTM that runs backwards you preserve information from the future and using the two hidden states combined you are able in any point in time to preserve information from both past and future.

What they are suited for is a very complicated question but BiLSTMs show very good results as they can understand context better

In [20]:
from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional
from keras.datasets import imdb
import numpy as np

top_words = 20000
maxlen = 80 
embedding_dims = 128
batch_size = 32

In [21]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=top_words)
print(len(x_train), 'train sequences')


25000 train sequences


In [22]:
print(len(x_test), 'test sequences')

25000 test sequences


In [23]:
print('x_train shape:', x_train.shape)


x_train shape: (25000,)


In [24]:
print('x_test shape:', x_test.shape)

x_test shape: (25000,)


In [25]:
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
y_train = np.array(y_train)
y_test = np.array(y_test)

x_train shape: (25000, 80)
x_test shape: (25000, 80)


In [26]:
model = Sequential()
model.add(Embedding(top_words, embedding_dims, input_length=maxlen))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile('adam', 'binary_crossentropy', metrics=['accuracy'])

print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=4,
          validation_data=[x_test, y_test])

Train...
Train on 25000 samples, validate on 25000 samples
Epoch 1/4
25000/25000 [==============================] - 311s - loss: 0.4300 - acc: 0.7962 - val_loss: 0.3543 - val_acc: 0.8448
Epoch 2/4
25000/25000 [==============================] - 303s - loss: 0.2480 - acc: 0.9010 - val_loss: 0.3788 - val_acc: 0.8345
Epoch 3/4
25000/25000 [==============================] - 349s - loss: 0.1594 - acc: 0.9396 - val_loss: 0.4236 - val_acc: 0.8268
Epoch 4/4
25000/25000 [==============================] - 338s - loss: 0.0903 - acc: 0.9671 - val_loss: 0.6999 - val_acc: 0.8171
